In [1]:
import warnings; warnings.simplefilter('ignore')
try:
    import google.colab
    IN_COLAB = True
    !mkdir data models images
    #!wget -P data/ https://raw.githubusercontent.com/taudataanalytics/Pedas-2025/master/data/Tech_stocks.csv
    #!pip install darts nolds tsfresh seaborn prophet plotly ipywidgets --q
except:
    IN_COLAB = False
    print("Running the code locally, please make sure all the python module versions agree with colab environment and all data/assets downloaded")

Running the code locally, please make sure all the python module versions agree with colab environment and all data/assets downloaded


In [2]:
import darts, numpy as np, pandas as pd, seaborn as sns
import os, matplotlib, matplotlib.pyplot as plt, difflib
from darts.metrics import smape, mape
from darts import TimeSeries

seed = 42
np.random.seed(seed)
"Done"

'Done'

In [3]:
gt = "data/gt-gaikindo-aug-2025.csv"
ts = "data/submissions-timeStamp.csv"
path_ = "data/submissions/"

files = os.listdir(path_)
gt = pd.read_csv(gt)
ts = pd.read_csv(ts, encoding='utf-8', encoding_errors='ignore')
ts["Timestamp"] = pd.to_datetime(ts["Timestamp"], errors="coerce")
print(ts.info())
print(files[:5])
gt.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Timestamp  55 non-null     datetime64[ns]
 1   Tim        55 non-null     object        
dtypes: datetime64[ns](1), object(1)
memory usage: 1012.0+ bytes
None
['abyz.csv', 'akalimitas.csv', 'algorangers.csv', 'apaia.csv', 'blacklist.csv']


,Brand,Sales
0,Daihatsu,9846
1,Honda,4375
2,Mitsubishi,8982
3,Suzuki,5911
4,Toyota,18328


In [4]:
ts.head()

,Timestamp,Tim
0,2025-08-31 17:59:00,AlgoRangers
1,2025-09-02 21:24:00,Data Crusaders
2,2025-09-03 15:48:00,Tim PNJ 2025
3,2025-09-04 19:47:00,MLSolo
4,2025-09-04 20:47:00,Hello World


In [5]:
ts["file"]=""
ts["smape"]=""
for i,d in ts.iterrows():
    kel = "".join([s for s in d.Tim if s.isalnum()]).lower()
    fileName = difflib.get_close_matches(kel, files, n=1)[0]
    ts.loc[i, "file"] = fileName
    df = pd.read_csv(path_+fileName, sep=None, engine="python")
    score = smape(TimeSeries.from_series(gt["Sales"]), TimeSeries.from_series(df.iloc[:, -1]))
    ts.loc[i, "smape"] = score 
    
ts = ts.sort_values(by=["smape", "Timestamp"], ascending=[True, True])

In [6]:
ts.head(15)

,Timestamp,Tim,file,smape
12,2025-09-05 23:28:00,Predik-Sip,prediksip.csv,19.162901
39,2025-09-06 22:59:00,NDTS TEAM,ndtsteam.csv,28.224009
13,2025-09-06 03:20:00,CoSight,cosight.csv,37.461427
18,2025-09-06 13:59:00,SatriaPeDaS,satriapedas.csv,37.661477
26,2025-09-06 19:16:00,Cybernetsec UNWAHAS,cybernetsecunwahas.csv,37.870658
42,2025-09-06 23:19:00,Machine Learning Darmajaya,machinelearningdarmajaya.csv,38.755547
47,2025-09-06 23:45:00,Hot Sauce,hotsauce.csv,39.718818
9,2025-09-05 19:27:00,Poros Nusantara,porosnusantara.csv,40.428473
44,2025-09-06 23:38:00,apa ia,apaia.csv,40.490153
3,2025-09-04 19:47:00,MLSolo,mlsolo.csv,41.437131


In [7]:
ts.to_csv("ScoreBoard-Penyisihan-PeDaS-2025.csv", index=False)